# 2.3.3.1 Textlicher Teil Bebauungsplan – OCR (GPT-4o)

* Zero-Shot, Chain-Of-Thought
* Best results for text_prompts: A2, B1, C1, D1
* Best results for img_prompts: B2, C2, D2, E3

### Setup

In [1]:
from utils.openai import OpenAI
from utils.parser import Parser
from utils.runner import Runner
from utils.pprint import pprint
import asyncio

instructions = "Du bist ein Assistent zur getreuen Wiedergabe von Informationen aus einem Bebauungsplan. Achte auf Vollständigkeit."
samantha = OpenAI(instructions)
parser = Parser()
runner = Runner()

In [2]:
pdf_path = "../data/raw/bpläne/3_xplanung/BP_872A_AUFST_TT.pdf"
pdf_prompts = parser.pdf2prompts(pdf_path)
print("Seitenzahl:", len(pdf_prompts))

Seitenzahl: 66


### A) OCR komplettes PDF

In [3]:
instruction = 'Extrahiere den kompletten Textinhalt. Output im LaTeX-Format.'
instruction_prompt = parser.text2prompts([instruction])

In [ ]:
# A1) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Komplettes PDF auf einmal einlesen. Ziel: Konsistentes Layout.
pdf_text = samantha.request([*pdf_prompts, *instruction_prompt])
pprint(pdf_text)

In [13]:
# A2) OCR via GPT-4o / Kompletter schriftlicher Teil
# Idee: Jede Seite einzeln verarbeiten. Ziel: Fokus maximieren und Context-Window klein halten + Seitenzahlen beleiben erhalten.
prompt_chain = list(map(lambda prompt: samantha.lambdaRequest([prompt, *instruction_prompt]), pdf_prompts))
msgbp872apdf = await asyncio.gather(*prompt_chain)
pprint(msgbp872apdf)
%store msgbp872apdf

```latex
\documentclass{article}
\usepackage[utf8]{inputenc}
\usepackage{graphicx}

\begin{document}

\begin{center}
    \includegraphics[width=0.2\textwidth]{logo.png} \\
    \vspace{1cm}
    \textbf{\huge Stadt Augsburg} \\
    \vspace{2cm}
    \textbf{\LARGE Aufstellung} \\
    \vspace{1cm}
    \textbf{\LARGE Bebauungsplan Nr. 872 A} \\
    \vspace{0.5cm}
    \textbf{\LARGE „Zwischen Waldstraße und Döllgaststraße“} \\
    \vspace{0.5cm}
    \textbf{\large mit integriertem Grünordnungsplan} \\
    \vspace{2cm}
    \textbf{\large Textteil} \\
    \vspace{2cm}
    \textbf{\large In Kraft getreten am:} \\
    \vspace{0.5cm}
    \textbf{\huge 23.02.2024} \\
\end{center}

\end{document}
```
#############################################
```latex
\documentclass{article}
\usepackage[utf8]{inputenc}

\begin{document}

\section*{Inhaltsverzeichnis}

\begin{enumerate}
    \item Inhaltsverzeichnis \dotfill 2
    \item Abkürzungen \dotfill 3
    \item Ermächtigungsgrundlage \dotfill 5
    \item A

### B) Art der baulichen Nutzung

1. text_prompts
2. img_prompts

In [8]:
%store -r msgbaunvo_art
context_art = parser.text2prompts([msgbaunvo_art])

In [7]:
# B1) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store
%store -r msgbp872apdf
data = msgbp872apdf
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, data, context_art)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

KeyboardInterrupt: 

In [ ]:
# B2) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Art der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung".'
msg233_art = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_art)
pprint(msg233_art)
%store msg233_art

### Art der baulichen Nutzung

#### § 4 Art der baulichen Nutzung
(1) Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.

**Zulässig sind:**
- Wohngebäude,
- Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe, die der Versorgung des Gebietes dienen,
- Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.

**Nicht zulässig sind:**
- Betriebe des Beherbergungsgewerbes,
- sonstige nicht störende Gewerbebetriebe,
- Anlagen für Verwaltungen,
- Gartenbaubetriebe,
- Tankstellen.

(2) Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

#### D.5.2.1. Art der baulichen Nutzung

##### D.5.2.1.1. Allgemeines Wohngebiet
- Das aktuelle Planungskonzept sieht vorwiegend eine wohnbauliche Nutzung innerhalb des Plangebietes vor.
- Flächen für den Gemeinbedarf mit der Zweckbestimmung „Gesundheit/Fürsorge“ werden künftig als allgemeines Wohngebiet (WA)

In [ ]:
# B3) Textinformationen: Art der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS
# Komplettes PDF
instruction = 'Extrahiere alle Informationen zum Thema "Art der baulichen Nutzung". Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>).'
messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_art], img_type="pdf")
pprint(messages)

### C) Maß der baulichen Nutzung

1. text_prompts
2. img_prompts

In [ ]:
%store -r msgbaunvo_maß
context_maß = parser.text2prompts([msgbaunvo_maß])

In [ ]:
# C2) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
results = samantha.similaritySearchWithContext(instruction, data, context_maß)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

In [19]:
# C3) Textinformationen: Maß der baulichen Nutzung – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Maß der baulichen Nutzung" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Maß der baulichen Nutzung".'
msg233_maß = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_maß)
pprint(msg233_maß)
%store msg233_maß

Hier sind alle Informationen zum Thema "Maß der baulichen Nutzung" aus den bereitgestellten Seiten des Bebauungsplans:

### § 5 Maß der baulichen Nutzung

1. **Das Maß der baulichen Nutzung** ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.

2. **In den allgemeinen Wohngebieten** darf die zulässige Grundflächenzahl (GRZ) durch die Grundflächen der in § 19 Abs. 4 BauNVO aufgeführten Anlagen jeweils bis zu folgender GRZ überschritten werden:
   - WA1: GRZ maximal 0,65
   - WA2: GRZ maximal 0,80
   - WA3: GRZ maximal 0,75
   - WA4: GRZ maximal 0,75

### D.5.2.1.2 Maß der baulichen Nutzung

- **Das Maß der baulichen Nutzung** wird im allgemeinen Wohngebiet durch die Festlegung der höchstzulässigen Grundflächenzahl (GRZ) und der höchstzulässigen Geschossfläche (GF), durch die Zahl der Vollgeschosse (Höchstmaß bzw. zwingend) und durch die Höhe der baulichen Anlagen (OK, Höchstmaß bzw. zwingend) innerhalb der einzelnen überbaubaren Grundstücksflächen (Baufelder) ausrei

### D) Bauweise, überbaubare Grundstücksflächen

1. text_prompts
2. img_prompts

In [ ]:
%store -r msgbaunvo_bg
context_bg = parser.text2prompts([msgbaunvo_bg])

In [ ]:
# D2) Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – TEXT_PROMPTS
# Vector Store

instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
results = samantha.similaritySearchWithContext(instruction, data, context_bg)
pprint(len(results[1])) # retrieved documents
pprint(results[0]) # message

In [20]:
# D3)Textinformationen: Bauweise, überbaubare Grundstücksflächen – WITH CONTEXT – IMG_PROMPTS (BEST RESULT)
# Vorgefilterte Seiten

# instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen" beinhalten. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'
# msg_pages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context], img_type="pdf")
# pprint(msg_pages)

pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all
instruction = 'Extrahiere alle Informationen zum Thema "Bauweise, überbaubare Grundstücksflächen".'
msg233_bg = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_bg)
pprint(msg233_bg)
%store msg233_bg

### § 7 Bauweise, Überbaubare Grundstücksflächen, Abstandsflächen

(1) Sofern erforderlich, ist die Bauweise in der Planzeichnung (Teil A) festgesetzt.

(2) Die überbaubaren Grundstücksflächen sind durch Baulinien und Baugrenzen in der Planzeichnung (Teil A) festgesetzt.

(3) Vorbauten dürfen die Baugrenzen bis zu 1,5 m auf einer Breite von maximal 5,0 m je Eingang überschreiten. Balkone dürfen mit Ausnahme des jeweils obersten Geschosses die Baugrenzen bis zu 0,90 m auf einer Breite von maximal 7,50 m bei Balkonen überschreiten. Zwischen den einzelnen Balkonen muss ein Abstand von mindestens 3,25 m eingehalten werden.

(4) Terrassen an Gebäuden dürfen sich bis zu einer Tiefe von 2,5 m auch auf Flächen außerhalb der überbaubaren Grundstücksflächen erstrecken. Sie dürfen außerhalb der überbaubaren Grundstücksflächen nicht durch Wintergärten oder ähnliches überbaut werden.

(5) Im Kronenbereich der in der Planzeichnung (Teil A) dargestellten und festgesetzten Bestandsgehölze sind die in 

### E) Alle Themen in einem Prompt

1. img_prompts
2. text_prompts (TODO)

In [ ]:
%store -r msgbaunvo
schema = '''
+----------------------------------------------+
| Art der baulichen Nutzung                    |
+----------------------------------------------+
| Grundflächenzahl (GRZ) | Geschossfläche (GF) |
+----------------------------------------------+
'''
context_all = parser.text2prompts([msgbaunvo, schema])

In [ ]:
# E1) Textinformationen – WITH CONTEXT
# Schema / BauNVO
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche.'

messages = await samantha.extractTextFromImagesWithContexts(instruction, [pdf_path], [context_all], img_type="pdf")
pprint(messages)

In [3]:
# E2) Ein Thread – Analyse und Extraktion in einem Schritt – WITHOUT CONTEXT
# Problem: Kontext ist sehr groß, sodass GPT den Überblick verliert und Informationen verliert. 
instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Liste am Ende die jeweiligen Seitenzahlen als Referenz auf. Output-Format: ###<Thema>: <Informationen> ### Referenzen: <Seitenzahlen>)'
messages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(messages)

### Art der baulichen Nutzung:
- Die in der Planzeichnung (Teil A) mit WA1 bis WA4 gekennzeichneten Bereiche werden als Allgemeines Wohngebiet gemäß § 4 BauNVO festgesetzt.
- Zulässig sind:
  - Wohngebäude,
  - Läden, Schank- und Speisewirtschaften sowie nicht störende Handwerksbetriebe,
  - Anlagen für kirchliche, kulturelle, soziale, gesundheitliche und sportliche Zwecke.
- Nicht zulässig sind:
  - Betriebe des Beherbergungsgewerbes,
  - sonstige nicht störende Gewerbebetriebe,
  - Anlagen für Verwaltungen,
  - Gartenbaubetriebe,
  - Tankstellen.
- Fremdwerbeanlagen sind im gesamten Plangebiet nicht zulässig.

### Maß der baulichen Nutzung:
- Das Maß der baulichen Nutzung ist in den Nutzungsschablonen in der Planzeichnung (Teil A) festgesetzt.
- Zulässige Grundflächenzahl (GRZ) gemäß § 19 Abs. 4 BauNVO:
  - WA1: GRZ maximal 0,65,
  - WA2: GRZ maximal 0,80,
  - WA3: GRZ maximal 0,75,
  - WA4: GRZ maximal 0,75.
- Die zulässigen Gebäudeoberkanten (OK) sind für die einzelnen Bereiche in 

In [14]:
# E3) Ein Thread – Relevante Seiten extrahieren (Vorverarbeitung) – (Bestes Ergebnis) – WITHOUT CONTEXT
# Wenn Dokument zu groß wird: z.B. chunk_size=10 pages, overlap=2 pages.
# chunk_size = 10, overlap = 2
# n = math.ceil(len(pdf_prompts)/chunk_size)
# chunks = list(map(lambda i: pdf_prompts[slice(max(i*chunk_size-overlap,0),i*chunk_size+chunk_size)], range(n)))
instruction = 'Die Bilder repräsentieren ein PDF Dokument. Extrahiere alle Seiten die Informationen zu folgenden Themen beinhalten: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche. JSON-Output-Format: {"Seiten": [<Seitenzahlen>]}'

msg_pages = await samantha.extractTextFromImage(instruction, pdf_path, img_type="pdf")
pprint(msg_pages)
pages = [6, 7, 8, 28, 29, 30, 31, 32, 33] # msg_pages_all

# instruction = 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise und überbaubare Grundstücksfläche.'
# msg233_all = samantha.extractTextFromFilteredPromptsWithContext(pages, pdf_prompts, instruction, context_all)
# pprint(msg233_all)
# %store msg233_all

```json
{
  "Seiten": [6, 7, 8, 28, 29, 30, 31, 32, 33]
}
```


In [7]:
# E4) Mehrere Threads – WITHOUT CONTEXT
# Idee: Jede Seite einzeln verarbeiten, um Context-Window klein zu halten.
# Problem: Informationen sind auf mehrere Seiten verteilt => fehlender Kontext => unvollständige Resultate.
async def run(pdf_prompt):
    return samantha.request([
        {
            "type": "text",
            "text": 'Extrahiere alle Informationen zu den folgenden Themen: Art der baulichen Nutzung, Maß der baulichen Nutzung, Bauweise, überbaubare Grundstücksfläche. Steht keine relevante Information zur Verfügung gebe "-" aus.'
        },
        pdf_prompt
    ])

prompt_chain = list(map(lambda pdf_prompt: run(pdf_prompt), pdf_prompts))
results = await asyncio.gather(*prompt_chain)
pprint(results)

Art der baulichen Nutzung: -
Maß der baulichen Nutzung: -
Bauweise: -
überbaubare Grundstücksfläche: -
Hier sind die extrahierten Informationen zu den angeforderten Themen aus dem Inhaltsverzeichnis des Bebauungsplans Nr. 872 A, Aufstellung:

- **Art der baulichen Nutzung**: -
- **Maß der baulichen Nutzung**: -
- **Bauweise**: -
- **Überbaubare Grundstücksfläche**: -

Das Inhaltsverzeichnis enthält keine spezifischen Informationen zu diesen Themen. Um detaillierte Informationen zu erhalten, müsste der vollständige Text des Bebauungsplans eingesehen werden.
Die Seite enthält nur eine Liste von Abkürzungen und deren Erläuterungen. Es sind keine Informationen zu den Themen "Art der baulichen Nutzung", "Maß der baulichen Nutzung", "Bauweise" und "überbaubare Grundstücksfläche" vorhanden.

- Art der baulichen Nutzung: -
- Maß der baulichen Nutzung: -
- Bauweise: -
- überbaubare Grundstücksfläche: -
- Art der baulichen Nutzung: WA (Allgemeines Wohngebiet)
- Maß der baulichen Nutzung: -
- Bau